In [250]:
import os
import csv
from bs4 import BeautifulSoup
import pandas as pd
import requests as rq
import time
import random
import unicodedata
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import *
import string
import re
from math import *
import json
from collections import defaultdict
from scipy import spatial
import heapq

In [4]:
path = os.getcwd()
movies = pd.DataFrame(pd.read_html(path + "\\movies1.html")[0])
movies.drop('Id', inplace=True, axis = 1)

In [111]:

# for i in range(len(movies)):
#     try:
#         response = rq.get(movies.URL[i])
#     except rq.exceptions.RequestException as e:
#         print(e)
#         time.sleep(20*60 + 30)
#         response = rq.get(movies.URL[i])
#     soup = BeautifulSoup(response.text, 'html.parser')
#     f = open('article_'+str(i)+'.html','w')
#     f.write(str(soup))
#     f.close()
#     time.sleep(random.choice(range(1,6)))

In [58]:
path1 = path + '\\Articles'
path1

'C:\\Users\\simon\\Desktop\\ADM\\Homework 3\\Articles'

In [415]:
# for i in range(30000):
#     article = open(path1+'\\article_'+str(i)+'.html', 'r')
#     soup = BeautifulSoup(article, 'html.parser')
#     d = {}
#     try:
#         for x in soup.find('table', class_="infobox vevent").find('th').find_all_next('th'):
#             d[x.text] = unicodedata.normalize('NFKD',x.next_sibling.get_text(separator = '<br/>').replace('<br/>', ',').strip())
#     except:
#         pass
#     title = str(soup.select('h1')[0].text)

#     start = soup.find('p')
#     intro = start.text.strip()
#     while len(intro) == 0:
#         start = start.find_next('p')
#         intro = start.text.strip()
#     for elem in start.next_siblings:
#         if elem.name != 'p':
#             break
#         intro += elem.text.strip()   

#     try:
#         start = soup.find('h2').find_next('p')
#         plot = start.text.strip()
#         for elem in start.next_siblings:
#             if elem.name != 'p':
#                 break
#             plot += elem.text.strip()

#     except:
#         plot = "NA"




#     try :
#             director = d['Directed by']
#     except: 
#             director = "NA"

#     try :
#             producer = d['Produced by']
#     except: 
#             producer = "NA"

#     try :

#             writer = d["Written by"]
#     except:
#             writer = "NA"


#     try :
#             starring = d["Starring"].strip()

#     except:
#             starring = "NA"

#     try :

#             music = d["Music by"]
#     except :
#             music = "NA"

#     try :
#             release_date = d["Release date"]

#     except :

#             release_date = "NA"

#     try :
#             run_time = d["Running time"]

#     except :
#             run_time = "NA"

#     try :
#             country = d["Country"]

#     except :

#             country = "NA"

#     try :
#             language = d["Language"]

#     except:
#             language = "NA"


#     try :
#             budget = d["Budget"]

#     except :
#             budget = "NA"


#     with open(path+"\\TSV\\article_" + str(i) + ".tsv", "w" ,encoding="utf-8") as out_file:
#             tsv_writer = csv.writer(out_file, delimiter='\t')
#             tsv_writer.writerow(['title', 'intro', 'plot', 'director', 'producer', 'writer' , 'starring', 'music', 'release_date','run_time', 'country' , 'language' , 'budget'])
#             tsv_writer.writerow([title, intro, plot, director, producer, writer , starring, music, release_date,run_time, country , language , budget])


In [59]:
df  = pd.DataFrame(pd.read_csv(path+"\\TSV\\article_" + str(i) + ".tsv", delimiter = '\t').iloc[:,0:3])
df

,title,intro,plot
0,Jack and the Beanstalk (1902 film),Jack and the Beanstalk is a 1902 American shor...,In this earliest known adaptation of the class...


In [60]:
path2 = path+'\\TSV\\'

In [61]:
stop_words = set(stopwords.words('english')) 
stemmer = PorterStemmer()

In [62]:
def clean(text):
    text = text.lower()
    words = word_tokenize(text) #devide the text into substrings
    filtered1 = [w for w in words if not w in stop_words] #remove stop words
    filtered2 = list(filter(lambda word: word not in string.punctuation, filtered1))
    filtered3 = []
    for word in filtered2:
        try:
            filtered3 += re.findall(r'\w+', word) 
        except:
            pass
    
    filtered3 = [stemmer.stem(w) for w in filtered3] #stemming
    filtered4 = [c.replace("''", "").replace("``", "") for c in filtered3 ] #removing useless '' and  `` characters
    filtered4 = [f for f in filtered4 if len(f)>1]
    return filtered4

In [588]:
allwords = list()
docwords = dict()
tsvs = dict()
for i in range(0,30000):
    with open(path2+"article_" + str(i) + ".tsv", encoding = "utf-8") as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        for row in rd:
            if row :
                tsv = row
    text = ' '.join([tsv[1],tsv[2]])
    tsvs[i] = tsv
    cleared = clear(text)
        
    docwords['document_'+str(i)] = cleared
    allwords += cleared

In [279]:
tsvs = dict()
for i in range(0,30000):
    with open(path2+"article_" + str(i) + ".tsv", encoding = "utf-8") as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        for row in rd:
            if row :
                tsv = row
    tsvs[i] = tsv

In [285]:
savetojson(path+"\\tsvs.json", tsvs)

In [284]:
path

'C:\\Users\\simon\\Desktop\\ADM\\Homework 3'

In [581]:
vocabulary = {}   
allwords = list(set(allwords))
for i in range(len(allwords)):
    vocabulary[str(i)] = allwords[i]

In [613]:
reverse_voc = {v:k for k,v in vocabulary.items()}

In [ ]:
from collections import defaultdict

inverted = defaultdict(list)
for doc in docwords.keys():
    for word in docwords[doc]:
        if not doc in inverted[reverse_voc[word]]:
            inverted[reverse_voc[word]].append(doc)

In [161]:
def savetojson(pathfile, obj):
    with open(pathfile, "w" ,encoding="utf-8") as out_file:
        out_file.write(json.dumps(obj, ensure_ascii = False))
        out_file.close()

In [ ]:
savetojson(path + "\\WORDS\\DocWords.json", docwords)
savetojson(path + "\\WORDS\\vocabulary.json", vocabulary)
savetojson(path + "\\WORDS\\Inverted_index.json", inverted)

# Search Engine

In [63]:
query = input("Insert your query: ")

Insert your query: disney 2019


In [64]:
query = clean(query)
query

['disney', '2019']

In [67]:
with open(path + "\\WORDS\\vocabulary.json", encoding = "utf-8") as fd:
        vocabulary = json.load(fd)
with open(path + "\\WORDS\\Inverted_index.json", encoding = "utf-8") as fd:
        inverted_index = json.load(fd)
reverse_voc = reverse_voc = {v:k for k,v in vocabulary.items()}

In [68]:
#now we define a function to get the query and return term_id of each vocab in query if they exist in vocabulary dataset
def get_query_index(query) :
    indexes = []
    for i in range(len(query)) :
        if query[i] in vocabulary.values() : #if the vocab in query exist in vocabulary dataset
            indexes.append(reverse_voc[query[i]]) #add term_id of that vocab to query

        else : #if it does not exist in vocabulary we replace it with 0
            indexes.append('NA')
    return(indexes)


Now we write a function to execute the query. First we should build a list that contains set of inverted_indexes of query elements and then since we want documents that contain all words of query we should find the intesection of these sets

In [69]:
def execute_query(query):
    query = get_query_index(query)
    docs = []
    for i in query :
        if (i == 'NA') : 
#if there is a vocab in query that does not exist in vocabulary dataset, there isn't a match and we should terminate the function
            return("No match for your query")
        else :
            docs.append(set(inverted_index[i]))
        
    docs = set.intersection(*docs)
    return(docs)

In [77]:
#Creating URL Database
Movies = dict()
c = 0
for i in range(1,4):
    movies = pd.DataFrame(pd.read_html(path + "\\movies" + str(i) + ".html")[0])
    movies.drop('Id', inplace=True, axis = 1)
    Urls = list(movies['URL'])
    for i in range(len(Urls)):
        Movies[str(10000*c + i)] = Urls[i]
    c += 1
        

In [310]:
savetojson(path+'\\AllMovies.json', Movies)

In [87]:
pd.DataFrame.from_dict(Movies, columns=['URL'], orient='index').tail()

,URL
29995,https://en.wikipedia.org/wiki/Paarai_(film)
29996,https://en.wikipedia.org/wiki/Jayam_(2003_film)
29997,https://en.wikipedia.org/wiki/Kaadhal_Kondein
29998,https://en.wikipedia.org/wiki/Kovilpatti_Veera...
29999,https://en.wikipedia.org/wiki/Whistle_(2003_film)


In [147]:
def Linked_URL(val): #we will use this to make the urls in output clickable
        # target _blank to open new window
        return '<a target="_blank" href="{}">{}</a>'.format(val, val)
    
def replacer(val):      #This is used to escape the character $ in the output for Intro,
    return val.replace('$', '\$')      #otherwise it would be interpreted by displayer

In [200]:
def get_results(query):
    results = []
    for file in execute_query(query):
        docid = file.split('_')[1]
        tsv = []
        with open(path2+"article_" + docid + ".tsv", encoding = "utf-8") as fd:
            rd = csv.reader(fd, delimiter="\t", quotechar='"')
            for row in rd:
                if row :
                    tsv.append(row)
        data = tsv[1]
        results.append([docid,data[0],data[1],Movies[docid]])  #create movies file before
        opentsv.close()
    result = pd.DataFrame(results, columns = ['Id','Title', 'Intro', 'Wikipedia Url'])
    return result

In [353]:
result = get_results(query)
result.head().style.format({'Wikipedia Url': Linked_URL}).format({'Intro': replacer})  # to get clickable urls     


,Id,Title,Intro,Wikipedia Url
0,12175,Little Odessa (film),"Little Odessa is a 1995 American crime drama film written and directed by James Gray, in his directorial debut, and starring Tim Roth, Edward Furlong, Moira Kelly, Maximilian Schell and Vanessa Redgrave. The film earned a Silver Lion at the Venice Film Festival and the prestigious Grand Prix of the Belgian Film Critics Association. It also earned admiration from French master Claude Chabrol.[2]",https://en.wikipedia.org/wiki/Little_Odessa_(film)
1,27019,Gajakesari,"Gajakesari is a 2014 Indian Kannada epic historical action film directed by cinematographer Krishna and produced by Jayanna and Bogendra. The film stars Yash in leading role along with Amulya and Anant Nag. The film also prominently features an elephant named Arjuna from Waynad, Kerala.[1] Actor Prakash Raj plays as narrator for the film to introduce Yash's character. Also uncredited writer Sai Prasad did the still photography and concept design for the making of the movie. This film is also dubbed in Hindi as Gajakeasari:The Big Lion.[2]",https://en.wikipedia.org/wiki/Gajakesari
2,13734,Black Knight (film),"Black Knight is a 2001 American fantasy adventure comedy film starring Martin Lawrence and directed by Gil Junger. In addition to Lawrence, Black Knight had a supporting cast of Marsha Thomason, Tom Wilkinson, Vincent Regan, and Kevin Conway.In the film, Lawrence plays Jamal, a theme park employee who is transported through time to medieval England. The film was shot at various locations in North Carolina, mainly Wilmington and Carolina Beach.The film was released in November 2001 and went on to gross \$39,976,235[1] at the worldwide box office, a loss from its \$50 million budget. Its critical reception was poor.",https://en.wikipedia.org/wiki/Black_Knight_(film)
3,20568,To Walk with Lions,"To Walk with Lions is a 1999 film directed by Carl Schultz and starring Richard Harris as George Adamson and John Michie as Tony Fitzjohn. It follows the later years of wild game preserver/naturalist Adamson.After his marriage to Joy Adamson of Born Free fame, Adamson spent the latter part of his life protecting the lions and other wildlife in the Kora National Reserve, Kenya. He encounters poachers and government corruption blocking his quest for wildlife preservation.",https://en.wikipedia.org/wiki/To_Walk_with_Lions
4,25122,Apoorva Sagodharargal (1989 film),"Apoorva Sagodharargal (English: Rare Brothers) is a 1989 Indian Tamil-language comedy-drama film, directed by Singeetam Srinivasa Rao. The film stars Kamal Haasan in three roles, as a mechanic, a circus clown who is a dwarf and a police officer, along with Gautami and Rupini playing lead roles with Srividya, Manorama, Nagesh, Jaishankar, Nassar, and Delhi Ganesh playing supporting roles. The music for the film was composed by Ilaiyaraaja, with lyrics written by Vaali. The film was produced by Kamal Haasan and his brother Chandra Haasan under the production company Raaj Kamal Films International.The film tells the story of twins who were separated during childhood who grow up in different localities, with one of them Appu, being a midget working for a circus. Appu comes to know that his father was an honest police officer who was killed by four criminals. The rest of the film shows how Appu succeeds in avenging the death of his father and how Raja proves his innocence as he is blamed for the murders committed by Appu.The film's story was written by Panchu Arunachalam, while Kamal Haasan and Crazy Mohan wrote screenplay and dialogues respectively. B. Lenin and V. T. Vijayan handled editing respectively, while the cinematography was handled by P. C. Sriram. The film premiered at the International Film Festival of India, The film was released on 14 April 1989, was a box office success and completed a 200-day run. It became the highest grossing Tamil cinema movie, until its collections were broken by Baashha (1995). It was dubbed into Telugu as Vichithra Sodarulu 

['lion']

# TFIDF Inverted Index

In [149]:
with open(path + "\\WORDS\\Inverted_index.json", encoding = "utf-8") as fd:
        inverted_index = json.load(fd)

In [151]:
with open(path + "\\WORDS\\DocWords.json", encoding = "utf-8") as fd:
        docwords = json.load(fd)

In [321]:
IDFs = dict()
inv_ind_tfIDF = defaultdict(list)
for term in inverted_index.keys() :
        IDFs[term] = log(1+ 30000/len(inverted_index[term]))
        for doc in inverted_index[term] :
            tf = docwords[doc].count(vocabulary[term]) / len(docwords[doc])
            tfidf = tf * IDFs[term]
            inv_ind_tfIDF[term].append((doc,round(tfidf, 3)))
                       

In [162]:
savetojson(path + "\\WORDS\\TfIdf_inv_index.json", inv_ind_tfIDF)

# 2.2) Conjunctive query & Ranking score

In [254]:
cosine_distance = spatial.distance.cosine
def cosine_similarity(a,b):
    return 1 - cosine_distance(a,b)

In [386]:
def querytf(query):
    qtf = defaultdict()
    for word in query :
        term_id = reverse_voc[word]
        try :
            qtf[term_id] += 1/len(query) 
            
        except :
            
            qtf[term_id] = 1/len(query)
    for term in qtf.keys():
        qtf[term] *= IDFs[term]
        
    return(qtf)

In [322]:
vocabulary = json.loads(open(path + "\\WORDS\\vocabulary.json", encoding = 'utf-8').read())
inv_ind_tfIDF = json.loads(open(path + "\\WORDS\\TfIdf_inv_index.json", encoding = 'utf-8').read())

In [379]:
def execute_SE2(query) :
    results = get_results(query)
    ids = get_query_index(query)
    wordtf = defaultdict(list)
    for i in results['Id']:
        doc = 'document_'+str(i)
        for term in ids:
            for docs in inv_ind_tfIDF[term]:
                if docs[0] == doc:
                    wordtf[i].append(docs[1])
        
    return(wordtf)


In [ ]:
with open(path + "\\tsvs.json", encoding = "utf-8") as fd:
        newdict = json.load(fd)

In [389]:
query = input("insert your text : ")

query = clean(query)

wordtf = execute_SE2(query)
qtfs = querytf(query)
list_query = list(qtfs.values())

heap = []
for doc in wordtf:
    heapq.heappush(heap, (cosine_similarity(list_query, wordtf[doc]), doc))
    

    
heap_result = heapq.nlargest(10, heap)
df = dict()
for x,y in heap_result:
    df[y] = newdict[y][0:2]
    df[y].append(Movies[y])
    df[y].append(x)
     # to get clickable urls
df = pd.DataFrame.from_dict(df, orient = 'index', columns=['Title', 'Intro', 'URL', 'Score'])
df.style.format({'URL': make_clickable}).format({'Intro' : replacer})    





insert your text : lion king disney


,Title,Intro,URL,Score
13043,RocketMan,"RocketMan (also written as Rocket Man) is a 1997 American comic science fiction film directed by Stuart Gillard and starring Harland Williams, Jessica Lundy, William Sadler and Jeffrey DeMunn. It was produced by Walt Disney Pictures and Caravan Pictures, and was released on October 10, 1997.",https://en.wikipedia.org/wiki/RocketMan,0.999869
12927,George of the Jungle (film),"George of the Jungle is a 1997 American romantic comedy film directed by Sam Weisman and based on the Jay Ward cartoon of the same name, which is also a spoof of Tarzan. The film was produced by Walt Disney Pictures with Mandeville Films and The Kerner Entertainment Company and was released in theatres on July 16, 1997. It stars Brendan Fraser as the eponymous main character, a primitive man who was raised by animals in an African jungle; Leslie Mann as his love interest; and Thomas Haden Church as her treacherous fiancé.A direct-to-video sequel, George of the Jungle 2, was released on DVD in 2003; however, only three of the original actors (Thomas Haden Church, John Cleese, and Keith Scott) returned for the sequel.",https://en.wikipedia.org/wiki/George_of_the_Jungle_(film),0.999869
21519,The Care Bears Movie,"The Care Bears Movie is a 1985 Canadian-American animated fantasy film and the second feature film from the Canadian animation studio Nelvana (after the 1983 film, Rock & Rule). One of the first films based directly on a toy line, it introduced the Care Bears characters and their companions the Care Bear Cousins.[nb 2] In the film, an orphanage owner (Mickey Rooney) tells a story about the Care Bears, who live in a cloud-filled land called Care-a-Lot. Traveling across Earth, the Bears help two lonely children named Kim and Jason, who lost their parents in a car accident, and also save a young magician's apprentice named Nicholas from an evil spirit's influence. Deep within a place called the Forest of Feelings, Kim, Jason, and their friends soon meet another group of creatures, the Care Bear Cousins.American Greetings, the owners of the Care Bears characters, began development of a feature film adaptation in 1981. Later on, the card company chose Nelvana to produce it, granted them rights to the characters, and financed the film along with cereal manufacturer General Mills and television syndicator LBS Communications. Nelvana's founders served as producers, while fellow employee Arna Selznick directed the film. Production lasted eight months, with a production budget of at least \$2 million, and took place in Canada, Taiwan, and South Korea. The voice cast included Mickey Rooney, Georgia Engel, Jackie Burroughs and Cree Summer. Two pop music stars, Carole King and John Sebastian, contributed several songs. Although major American film studios passed on the project, newly established independent distributor The Samuel Goldwyn Company acquired the distribution rights to the film and soon spent a record \$24 million promoting it.The film premiered on March 24, 1985, in Washington, D.C., and was released in North America on March 29, 1985.[nb 3] Another Nelvana work, Strawberry Shortcake Meets the Berrykins, played alongside the feature in theatres. The Care Bears Movie received mixed reviews from critics, who raised concern over its potential as a full-length advertisement for the title characters among other aspects. It went on to earn \$23 million domestically; as Canada's highest-grossing film during 1985 (with C\$1.845 million), it won a Golden Reel Award. With over \$34 million in worldwide sales, it set a box-office record for Canadian and non-Disney animation, and has remained one of Goldwyn's largest-earning releases. The film's success saved Nelvana from closing and helped revive films aimed at children in the US market. It has since been cited as inspiring a spate of toy-based animated and live-action features. It was soon followed by two sequels, A New Generation (1986) and Adventure in Wonderland (1987

[1.5329187392593246, 0.962692185457649, 1.4026825549898354]

In [255]:
cosine_similarity(list_query, wordtf['12282'])

1.0

In [246]:
wordtf['12282']
list_query

[1.0]

In [235]:
execute(query)

defaultdict(list,
            {'12282': [0.007],
             '10261': [0.153],
             '15530': [0.025],
             '21994': [0.021],
             '17141': [0.007],
             '9706': [0.012],
             '20866': [0.012],
             '8386': [0.054],
             '2892': [0.022],
             '12943': [0.013],
             '13336': [0.051],
             '16819': [0.015],
             '15005': [0.009],
             '9070': [0.015],
             '14358': [0.009],
             '14979': [0.009],
             '14867': [0.009],
             '13043': [0.009],
             '9365': [0.038],
             '13497': [0.019],
             '12060': [0.02],
             '7957': [0.021],
             '13573': [0.04],
             '7576': [0.032],
             '14774': [0.035],
             '4440': [0.08],
             '15872': [0.02],
             '3829': [0.03],
             '8300': [0.028],
             '14195': [0.023],
             '16016': [0.007],
             '15321': [0.039],
     

In [175]:
query = input("insert your text : ")

query = clean(query)

insert your text : disney movie


In [180]:
querytf(query)

{'95223': 0.5, '28257': 0.5}

In [469]:
with open(path2+'article_15048.tsv', encoding = 'utf8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        if row :
            tsv = row
            
d = {}

if tsv[6] == 'NA':
    d['Starring'] = '0'
else:
    d['Starring'] = str(len(tsv[6].split(',')))

try:
    d['Release Year'] = re.search(r'\d{4}', tsv[8]).group(0)
except:
    d['Release Year'] = '0'
    
try:
    d['Runtime']      = re.search(r'\d+.*',tsv[9]).group(0)
except:
    d['Runtime']    = '0'

#some movies have running time expressed in reels, and the conversion in minutes is not univoque, so we'll just ignore those info
if re.search(r'min', d['Runtime']):
    d['Runtime'] = re.search(r'\d+[\.|\,|:]*\d*', d['Runtime']).group(0)
    d['Runtime'] = re.search(r'\d+', d['Runtime']).group(0)
else:
    d['Runtime'] = '0'
    
try:
    d['Budget']   = re.findall(r'\$.*', tsv[12])[0]
except:
    d['Budget']  = '0'

if re.search(r'mil', d['Budget']):
    d['Budget']  = str(int(float(re.search(r'\d+[\.|\,]*\d*', d['Budget']).group(0))*10**6))
elif re.search(r',', d['Budget']):
    d['Budget'] = d['Budget'].replace(',', '').replace('$', '')
else:
    d['Budget'] = d['Budget'].replace('.', '').replace('$', '')

d

{'Starring': '9',
 'Release Year': '2007',
 'Runtime': '122',
 'Budget': '55000000'}

'document_0'

In [657]:
savetojson(path+"\\WORDS\\tfidf_index.json", inv_ind_tfIDF)

In [537]:
q = dict()
q['runtime'] = 150
Runtimes = [6,12,15,57,100,132]
minrun = min(Runtimes)
maxrun = max(Runtimes)
if d['Runtime'] != 0:
    runscore = 1 - abs(int(q['runtime'])- int(d['Runtime']))/(maxrun - minrun)
else:
    runscore = 0
runscore

maxbud = 400000000
minbud = 500
q['Budget'] = "expensive"

if q['Budget'] == 'expensive':
    bud_score = (maxbud - int(d['Budget']))/(maxbud-minbud)
elif q['Budget'] == 'inexpensive':
    bud_score = (int(d['Budget']) - minbud)/(maxbud-minbud)
    
bud_score

0.8625010781263477

In [ ]:
q[]

In [2]:
q = dict()
oldnew = input("Do you prefer an old movie or a new-released movie? Please type O for Old and N for New:")
if oldnew == "O" :
    q["release"] = "O"
if oldnew == "N" :
    q["release"] = "N"
    
    
    
year = input("Do you want to specify the release year ?Please type Y for Yes and N for No: ")
if year == "N" :
    q["year"] = "NA"
if year == "Y" :
    year = input("please specify the release date") 
    q["year"] = year
    
    

Runtime = input("Do you want to specify the length of the movie? Please type Y for Yes and N for No:")
if Runtime == "N" :
    q["Runtime"] = "NA"
if Runtime == "Y" :
    Runtime = input("please specify the length of the movie")
    q["Runtime"] = Runtime
    
    
starring = input("Is number of stars an important factor for you? Please type Y for Yes and N for No:")
q["starring"] = starring


budget = input("Is movie budget an important factor for you? Please type Y for Yes and N for No:")
q["budget"] = budget


Do you prefer an old movie or a new-released movie? Please type O for Old and N for New:O
Do you want to specify the release year ?Please type Y for Yes and N for No: Y
please specify the release date1924
Do you want to specify the length of the movie? Please type Y for Yes and N for No:Y
please specify the length of the movie120
Is number of stars an important factor for you? Please type Y for Yes and N for No:Y
Is movie budget an important factor for you? Please type Y for Yes and N for No:10
